In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, split, lit,col


In [12]:
df_medals = spark.read.option("header", "true")\
                        .option("inferSchema", "true")\
                        .csv("/home/iceberg/data/medals.csv")

df_medalsMatchesPlayers = spark.read.option("header", "true")\
                        .option("inferSchema", "true")\
                        .csv("/home/iceberg/data/medals_matches_players.csv")

df_explicitBroadcastMedalsMatchesPlayers = df_medalsMatchesPlayers.join(broadcast(df_medals), "medal_id","inner") 

In [13]:
df_maps = spark.read.option("header", "true")\
                       .option("inferSchema", "true")\
                        .csv("/home/iceberg/data/maps.csv")

df_matches = spark.read.option("header", "true")\
                        .option("inferSchema", "true")\
                        .csv("/home/iceberg/data/matches.csv")

df_explicitBroadcastMatchesMaps = df_matches.join(broadcast(df_maps), "mapid", "inner")  

In [ ]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
matches_bucketed_DDL = """
 CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """
spark.sql(matches_bucketed_DDL)

df_matches.select(
     col("match_id"), col("mapid"), col("is_team_game"), col("playlist_id"), col("completion_date"))\
     .write.mode("overwrite")\
    .partitionBy("completion_date")\
     .bucketBy(16, "match_id")\
    .saveAsTable("bootcamp.match_details_bucketed")

In [ ]:

df_matchDetails =  spark.read.option("header", "true")\
                        .option("inferSchema", "true")\
                        .csv("/home/iceberg/data/match_details.csv")

spark.sql("""DROP TABLE IF EXISTS bootcamp.match_details_bucketed""")
matchDetailBucketedDDL = """
 CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """
spark.sql(matchDetailBucketedDDL)

df_matchDetails.select(
     col("match_id"), col("player_gamertag"), col("player_total_kills"),col("player_total_deaths"))\
     .write.mode("overwrite")\
   .bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

In [ ]:
# medals Matches Players

spark.sql("""DROP TABLE IF EXISTS bootcamp.medals_match_players_bucketed""")
medalsMatchesPlayers = """
 CREATE TABLE IF NOT EXISTS bootcamp.medals_match_players_bucketed (
     match_id STRING,
     player_gamertag STRING,
     medal_id STRING,
     count INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """
spark.sql(medalsMatchesPlayers)

df_medalsMatchesPlayers.select(
     col("match_id"), col("player_gamertag"), col("medal_id"), col("count"))\
     .write.mode("overwrite")\
   .bucketBy(16, "match_id").saveAsTable("bootcamp.medals_match_players_bucketed")